In [ ]:
!pip install yfinance==0.1.67


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.38
    Uninstalling yfinance-0.2.38:
      Successfully uninstalled yfinance-0.2.38


In [ ]:
import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
import warnings
# Ignore all warnings
warnings.filterwarnings("ignore", category=FutureWarning)

## Load and prepare the data.
Load your dataset containing stock symbols and dates from a CSV file into a pandas DataFrame.
Convert the 'date' column to datetime objects to facilitate date manipulation.


In [ ]:
import pandas as pd
import numpy as np
data=pd.read_csv('raw_analyst_ratings.csv')

In [ ]:
data['date'] = pd.to_datetime(data['date'], errors='coerce').dt.date

In [ ]:
data.head(10)

,Unnamed: 0,headline,url,publisher,date,stock
0,0,Stocks That Hit 52-Week Highs On Friday,https://www.benzinga.com/news/20/06/16190091/s...,Benzinga Insights,2020-06-05,A
1,1,Stocks That Hit 52-Week Highs On Wednesday,https://www.benzinga.com/news/20/06/16170189/s...,Benzinga Insights,2020-06-03,A
2,2,71 Biggest Movers From Friday,https://www.benzinga.com/news/20/05/16103463/7...,Lisa Levin,2020-05-26,A
3,3,46 Stocks Moving In Friday's Mid-Day Session,https://www.benzinga.com/news/20/05/16095921/4...,Lisa Levin,2020-05-22,A
4,4,B of A Securities Maintains Neutral on Agilent...,https://www.benzinga.com/news/20/05/16095304/b...,Vick Meyer,2020-05-22,A
5,5,"CFRA Maintains Hold on Agilent Technologies, L...",https://www.benzinga.com/news/20/05/16095163/c...,vishwanath@benzinga.com,2020-05-22,A
6,6,"UBS Maintains Neutral on Agilent Technologies,...",https://www.benzinga.com/news/20/05/16094027/u...,vishwanath@benzinga.com,2020-05-22,A
7,7,Agilent Technologies shares are trading higher...,https://www.benzinga.com/wiim/20/05/16093805/a...,Benzinga Newsdesk,2020-05-22,A
8,8,Wells Fargo Maintains Overweight on Agilent Te...,https://www.benzinga.com/news/20/05/16093505/w...,vishwanath@benzinga.com,2020-05-22,A
9,9,10 Biggest Price Target Changes For Friday,https://www.benzinga.com/analyst-ratings/price...,Lisa Levin,2020-05-22,A


In [ ]:

# Convert 'date' column to pandas Timestamp objects
data['date'] = pd.to_datetime(data['date'])

# Now calculate the earliest and latest date for each stock
earliest_date = data.groupby('stock')['date'].min()
latest_date = data.groupby('stock')['date'].max()


In [ ]:
print(earliest_date)
print(latest_date)

stock
A      2020-05-22
AA     2020-05-18
AAC    2019-08-30
AADR   2013-03-05
AAL    2020-06-08
          ...    
AE     2019-05-08
AEB    2013-10-01
AEC    2015-01-28
AEE    2020-04-15
AEG    2020-05-15
Name: date, Length: 110, dtype: datetime64[ns]
stock
A      2020-06-05
AA     2020-06-09
AAC    2019-10-25
AADR   2013-09-06
AAL    2020-06-10
          ...    
AE     2020-05-18
AEB    2013-10-01
AEC    2015-06-08
AEE    2020-06-09
AEG    2020-06-05
Name: date, Length: 110, dtype: datetime64[ns]


In [21]:
date_ranges = pd.DataFrame({
    'earliest_date': data.groupby('stock')['date'].min(),
    'latest_date': data.groupby('stock')['date'].max()

})
date_ranges.head()

,earliest_date,latest_date
stock,,
A,2020-05-22,2020-06-05
AA,2020-05-18,2020-06-09
AAC,2019-08-30,2019-10-25
AADR,2013-03-05,2013-09-06
AAL,2020-06-08,2020-06-10


In [22]:
date_ranges['date_range']=date_ranges['latest_date']-date_ranges['earliest_date']

In [23]:
date_ranges.head()

,earliest_date,latest_date,date_range
stock,,,
A,2020-05-22,2020-06-05,14 days
AA,2020-05-18,2020-06-09,22 days
AAC,2019-08-30,2019-10-25,56 days
AADR,2013-03-05,2013-09-06,185 days
AAL,2020-06-08,2020-06-10,2 days


In [ ]:
import pandas as pd
import yfinance as yf

# Load your dataset containing stock symbols and dates from a CSV file into a pandas DataFrame.
df = pd.read_csv('raw_analyst_ratings.csv')

# Convert the 'date' column to datetime objects to facilitate date manipulation.
df['date'] = pd.to_datetime(df['date'], errors='coerce')

# Determine date ranges and fetch stock data.
# Calculate the earliest and latest date for each stock to define the period for which to fetch stock prices.
date_ranges = df.groupby('stock')['date'].agg(['min', 'max'])

# Use yfinance to download stock data for each stock based on the calculated date ranges.
stock_data = {}
for stock in date_ranges.index:
    start_date = pd.to_datetime(date_ranges.loc[stock, 'min']).strftime('%Y-%m-%d')
    end_date = pd.to_datetime(date_ranges.loc[stock, 'max']).strftime('%Y-%m-%d')
    stock_data[stock] = yf.download(stock, start=start_date, end=end_date)

# Now, `stock_data` is a dictionary where each key is a stock symbol and the value is a DataFrame with the stock data.
